In [ ]:
import os, sys, pathlib, ipynbname, glob, shutil
import ansys.edb

from ansys.edb.core.database import *
from ansys.edb.core.definition import *
from ansys.edb.core.geometry import *
from ansys.edb.core.hierarchy import *
from ansys.edb.core.layer import *
from ansys.edb.core.layout import *
from ansys.edb.core.layout_instance import *
from ansys.edb.core.net import *
from ansys.edb.core.primitive import *
from ansys.edb.core.simulation_setup import *
from ansys.edb.core.terminal import *
from ansys.edb.core.utility import *
from ansys.edb.core.session import launch_session

In [ ]:
def copy_input():
    if os.path.exists(get_output_filename()):
        shutil.rmtree(get_output_filename())
    shutil.copytree(get_input_filename(), get_output_filename())
    return get_output_filename()

def get_fixture_file(filename):
    example_name = ipynbname.name()
    return os.path.join(os.getcwd(), 'fixtures', example_name, filename)
    
def get_input_filename():
    example_name = ipynbname.name()
    return os.path.join(os.getcwd(), f'{example_name}.aedb')
    
def get_output_filename():
    example_name = ipynbname.name()
    return os.path.join(get_output_dir(), f'{example_name}.aedb')

def get_output_files():
    example_name = ipynbname.name()
    return os.path.join(get_output_dir(), f'{example_name}.*')

def get_output_dir():
    return os.path.join(os.getcwd(), "results")

In [ ]:
# this will wipe away existing db/session if user runs this script multiple times in same jupyter kernel session.
%run helpers/example_teardown.ipynb

if 'RPC_SERVER' in os.environ:
    RPC_SERVER = os.environ['RPC_SERVER']
else:
    RPC_SERVER = None

session = launch_session(RPC_SERVER, 50051)

In [ ]:
if not pathlib.Path(get_output_dir()).is_dir():
    os.makedirs(get_output_dir())

for f in glob.glob(get_output_files()):
    if pathlib.Path(f).is_dir():
        shutil.rmtree(f)
    else:
        os.remove(f)

if pathlib.Path(get_input_filename()).is_dir():
    db = Database.open(copy_input(), False)
    cell = db.circuit_cells[0]
    layout = cell.layout
    print('db, cell, layout variables set to the default from edb design.')
else:
    db = Database.create(get_output_filename())
    cell = Cell.create(db, CellType.CIRCUIT_CELL, "Example")
    layout = cell.layout
    gnd_net = Net.create(layout, 'GND')
    print('clean design created with default db, cell, layout, gnd_net variables.')